In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import pandas as pd
from pandas_profiling import ProfileReport


import surprise
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise.model_selection import cross_validate
from surprise import KNNBaseline
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

import matplotlib.pyplot as plt
import mplcyberpunk
%matplotlib inline

# plt.style.use('ggplot')
plt.style.use("cyberpunk")

import os
os.getcwd()

'/home/jovyan/work/Capstone_3/Skincare-Recommender'

In [6]:
# Import Data
items = pd.read_csv('data/item_df.csv')
users = pd.read_csv('data/user_df.csv')

In [8]:
# Only parse out needed features
users = users[['user_id', 'item_name', 'star_rating']]

In [11]:
users[users['user_id'] == 'superkaylap']

,user_id,item_name,star_rating


In [14]:
users.head()

,user_id,item_name,star_rating
0,harleen253,item_1,5
1,supremekaylap15,item_1,2
2,Alybug97,item_1,4
3,Mikachew,item_1,2
4,jkbean,item_1,2


In [19]:
# Create profiling report
profile = ProfileReport(users, title='Users Pandas Profiling Report', explorative=True)
profile.to_file("data/users_profile.html")

In [ ]:
# #method 1 unigram
# for i in range(len(feature_list)):
#     chem = df2[df2.feature == feature_list[i]].chemicals
#     l = []
#     for c in chem:
#         l=l+tokenizer.tokenize(c.lower())
#         df_feature.chemicals_uni[i] = l

# #method 2 combination of unigram and bigram
# for i in range(len(feature_list)):
#     chem = df2[df2.feature == feature_list[i]].chemicals
#     l = []
#     for c in chem:
#         if ' ' in c:
#             bigrams = ngrams(c.lower().split(), 2)
#             l.extend(bigrams)
#         else:
#             l.append(c.lower())